In [61]:
import pandas as pd
import numpy as np
import requests
import datetime
from collections import defaultdict
import time
from bs4 import BeautifulSoup
from espn_api.hockey import League

In [59]:
url = "https://www.hockey-reference.com/leagues/NHL_2023_games.html"

header = {
"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
"X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

team_abbrev = {'Anaheim Ducks': 'ANA', 'Arizona Coyotes': 'ARI', 'Boston Bruins': 'BOS', 'Buffalo Sabres': 'BUF', 
               'Calgary Flames': 'CGY', 'Carolina Hurricanes': 'CAR', 'Columbus Blue Jackets': 'CBJ', 
               'Chicago Blackhawks': 'CHI', 'Colorado Avalanche': 'COL', 'Dallas Stars': 'DAL', 
               'Detroit Red Wings': 'DET', 'Edmonton Oilers': 'EDM', 'Florida Panthers': 'FLA', 
               'Los Angeles Kings': 'LAK', 'Minnesota Wild': 'MIN', 'Montreal Canadiens': 'MTL', 
               'Nashville Predators': 'NSH', 'New Jersey Devils': 'NJD', 'New York Islanders': 'NYI', 
               'New York Rangers': 'NYR', 'Ottawa Senators': 'OTT', 'Philadelphia Flyers': 'PHI', 
               'Pittsburgh Penguins': 'PIT', 'San Jose Sharks': 'SJS', 'Seattle Kraken': 'SEA', 
               'St. Louis Blues': 'STL', 'Tampa Bay Lightning': 'TBL', 'Toronto Maple Leafs': 'TOR', 
               'Vancouver Canucks': 'VAN', 'Vegas Golden Knights': 'VEG', 'Washington Capitals': 'WSH', 
               'Winnipeg Jets': 'WPG'}

league_id = 1062516540
year = 2023
swid = '{5EFFD991-59EF-46E0-BFD9-9159EF26E06C}'
espn_s2 = 'AEAl1p12J7wvYN4w3TRnw3OAIw1PpBg1b9JnL9U7WLqr2moKjdpSS1NzoLsjI%2FdBQmEYPNb8NCifAt3Wo24'\
            'eApiWFNml6Yqqo7EHYTwEeBQ%2FfxdQbRdeswVxGSZyoBPAUhXIR%2BUceG1xk1QdZyJri4NzQywdZJwceF8'\
                'va1K2Nt1e8GvnJFxUgHayGu73QXnPEBPcPG1DD5YgR0RBUDvIGCeo5KuUU7%2Bf58AhkAAPVsXvgl9sRX'\
                    'sMvyqaIIab9N7zRC8Us0vV94hBBvJ51OosG3qAQHMEA8ppRIng598cOjGJPQ%3D%3D'

In [3]:
df = pd.read_html(r.text)[0]

In [4]:
df = df[['Date', 'Visitor', 'Home']]

In [5]:
matchup_starts = [datetime.date(2022,10,7), datetime.date(2022,10,17)]

for i in range(1,22):
    matchup_starts.append(matchup_starts[i] + datetime.timedelta(weeks=1))
    
matchup_starts.remove(datetime.date(2023,2,6))
matchup_starts.append(datetime.date(2023,3,27))
    
# for date in matchup_starts:
#     print(date)

In [6]:
matchup_ends = []

for i in range(1, len(matchup_starts)):
    matchup_ends.append(matchup_starts[i] + datetime.timedelta(days=-1))
    
matchup_ends.append(datetime.date(2023,4,13))

# for date in matchup_ends:
#     print(date)

In [7]:
while True:
    try:
        matchup_input = int(input("Matchup Week: "))
    except ValueError:
        print("Please enter a valid week.")
        continue
    if matchup_input < 1:
        print("Please enter a valid week.")
        continue
    if matchup_input > 23:
        print("Please enter a valid week.")
        continue
    else:
        break

input_date_range = [matchup_starts[matchup_input - 1], matchup_ends[matchup_input - 1]]

Matchup Week: 15


In [8]:
mask = (df['Date'] >= str(input_date_range[0])) & (df['Date'] <= str(input_date_range[1]))
df = df.loc[mask]

In [9]:
team_count = defaultdict(int)

for i,r in df.iterrows():
    team_count[r['Home']] += 1
    team_count[r['Visitor']] += 1

In [10]:
team_count_df = pd.DataFrame.from_dict(dict(team_count), orient="index")
team_count_df = team_count_df.reset_index()
team_count_df.index = np.arange(1, len(team_count_df) + 1)
team_count_df = team_count_df.rename(columns={'index': 'Team', 0: 'Games'})

In [11]:
team_count_df = team_count_df.sort_values('Games', ascending=False)
team_count_df.index = np.arange(1, len(team_count_df) + 1)
team_count_df
# top10_df = team_count_df.head(n=10)
# top10_df

,Team,Games
1,Philadelphia Flyers,5
2,Boston Bruins,4
3,Pittsburgh Penguins,4
4,Winnipeg Jets,4
5,Chicago Blackhawks,4
6,Arizona Coyotes,4
7,Dallas Stars,4
8,Vegas Golden Knights,4
9,Ottawa Senators,4
10,Tampa Bay Lightning,4


In [30]:
def soup(abbr):
    soup = (
        BeautifulSoup(
            requests.get(
                'https://www.hockey-reference.com/teams/' + abbr + '/2023.html',
                headers=header,
            ).text,
            "lxml",
        ).find("table", {"id": "skaters"})
    )
    
    return soup

In [32]:
teams = []
for t in team_count_df['Team']:
#     print(team_abbrev[t])
    print(t)
    df = pd.read_html(str(soup(team_abbrev[t])), flavor="lxml")[0]
    df.columns = df.columns.droplevel(0)
    df.drop(df[df['Player'] == 'Team Total'].index, inplace = True)
    df.drop(df[df['Pos'] == 'G'].index, inplace = True)
    df = df[['Player', 'Pos', 'GP', 'G', 'A', 'PIM', 'PP', 'SH', 'S', 'BLK']]
    df.columns = ['Player', 'Pos', 'GP', 'G', 'A', 'PIM', 'PPG', 'PPA', 'SHG', 'SHA', 'S', 'BLK']
    ppp = df['PPG'] + df['PPA']
    shp = df['SHG'] + df['SHA']
    df.insert(8, "PPP", ppp, True)
    df.insert(11, "SHP", shp, True)
    df = df.drop(['PPG', 'PPA', 'SHG', 'SHA'], axis=1)
    teams.append(df)
    time.sleep(3.1)

Philadelphia Flyers
      Rk               Player   Age  Pos  GP    G    A  PTS   +/-  PIM  ...  \
0    1.0       Travis Konecny  25.0    C  37   24   22   46  -3.0   51  ...   
1    2.0          Kevin Hayes  30.0   RW  42   10   27   37  -6.0   15  ...   
2    3.0       Scott Laughton  28.0    C  39   11   14   25  -6.0   16  ...   
3    4.0         Joel Farabee  22.0   LW  43    9   16   25   4.0   23  ...   
4    5.0     Anthony DeAngelo  27.0    D  38    7   18   25  -8.0   38  ...   
5    6.0         Owen Tippett  23.0   RW  38   13   11   24  -9.0   12  ...   
6    7.0         Morgan Frost  23.0    C  42    8   12   20  -3.0   16  ...   
7    8.0   James van Riemsdyk  33.0   LW  23    7   11   18   0.0   12  ...   
8    9.0           Noah Cates  23.0   LW  43    5   12   17   5.0    2  ...   
9   10.0       Travis Sanheim  26.0    D  43    4   11   15  -1.0   26  ...   
10  11.0        Ivan Provorov  26.0    D  43    2   13   15  -7.0   12  ...   
11  12.0         Zack MacEwen  2

Pittsburgh Penguins
      Rk                 Player   Age  Pos  GP    G    A  PTS  +/-  PIM  ...  \
0    1.0          Sidney Crosby  35.0    C  42   21   27   48    1   22  ...   
1    2.0          Evgeni Malkin  36.0    C  42   14   25   39   -3   20  ...   
2    3.0          Jake Guentzel  28.0    C  38   17   20   37    1   28  ...   
3    4.0         Rickard Rakell  29.0   RW  42   15   11   26    3    6  ...   
4    5.0           Jason Zucker  31.0   LW  38   10   16   26    2   17  ...   
5    6.0             Bryan Rust  30.0   RW  42    9   17   26  -12   11  ...   
6    7.0            Jeff Carter  38.0    C  39    7   12   19   -3   16  ...   
7    8.0        Kasperi Kapanen  26.0   RW  33    6   11   17   -2    4  ...   
8    9.0            Kris Letang  35.0    D  29    2   14   16  -12   12  ...   
9   10.0           Brock McGinn  28.0   LW  42   10    5   15    2   12  ...   
10  11.0             Jeff Petry  35.0    D  28    3    9   12    4   18  ...   
11  12.0  Pierre-Oli

Chicago Blackhawks
      Rk               Player   Age  Pos  GP   G    A  PTS  +/-  PIM  ...  \
0    1.0             Max Domi  27.0   LW  41  13   17   30  -13   41  ...   
1    2.0         Patrick Kane  34.0   RW  38   8   20   28  -25   10  ...   
2    3.0       Jonathan Toews  34.0    C  40  12   12   24  -21   37  ...   
3    4.0       Taylor Raddysh  24.0   RW  41  12    7   19  -18    8  ...   
4    5.0           Seth Jones  28.0    D  31   2   14   16  -25   18  ...   
5    6.0     Philipp Kurashev  23.0    C  41   5   10   15  -26    8  ...   
6    7.0   Andreas Athanasiou  28.0    C  40   9    4   13  -15   26  ...   
7    8.0         Sam Lafferty  27.0    C  35   5    8   13   -9   17  ...   
8    9.0      Jason Dickinson  27.0    C  37   4    7   11  -14   14  ...   
9   10.0          Jake McCabe  29.0    D  38   2    9   11    4   19  ...   
10  11.0        Tyler Johnson  32.0    C  18   4    6   10   -4    8  ...   
11  12.0          Caleb Jones  25.0    D  32   1    8    

Dallas Stars
      Rk             Player   Age  Pos  GP    G    A  PTS  +/-  PIM  ...  \
0    1.0    Jason Robertson  23.0   LW  44   29   31   60   25    8  ...   
1    2.0        Roope Hintz  26.0   LW  40   19   25   44   23   12  ...   
2    3.0       Joe Pavelski  38.0    C  44   14   29   43   30    6  ...   
3    4.0         Jamie Benn  33.0   LW  44   16   22   38   13   16  ...   
4    5.0     Miro Heiskanen  23.0    D  41    7   26   33    7   16  ...   
5    6.0       Tyler Seguin  31.0    C  44   11   19   30    6   18  ...   
6    7.0    Mason Marchment  27.0    F  44    9   13   22   -1   58  ...   
7    8.0     Wyatt Johnston  19.0    C  44   11    7   18   -2   14  ...   
8    9.0      Ty Dellandrea  22.0    C  44    5   11   16    5   29  ...   
9   10.0        Esa Lindell  28.0    D  44    5    8   13   23    8  ...   
10  11.0     Nils Lundkvist  22.0    D  41    4    8   12   -6   26  ...   
11  12.0         Ryan Suter  38.0    D  44    0   12   12  -10   20  ...   

Ottawa Senators
      Rk                Player   Age  Pos  GP    G    A  PTS  +/-  PIM  ...  \
0    1.0         Brady Tkachuk  23.0   LW  42   15   26   41  -12   49  ...   
1    2.0           Tim Stützle  21.0   LW  38   18   21   39  -10   24  ...   
2    3.0         Claude Giroux  35.0    C  42   15   23   38   -2   12  ...   
3    4.0        Alex DeBrincat  25.0   RW  42   14   24   38  -15   12  ...   
4    5.0       Drake Batherson  24.0    C  42   13   24   37  -29   18  ...   
5    6.0         Thomas Chabot  26.0    D  37    7   15   22  -10   32  ...   
6    7.0           Shane Pinto  22.0    C  42   10    7   17  -16    8  ...   
7    8.0        Jake Sanderson  20.0    D  42    2   14   16   -4    4  ...   
8    9.0       Derick Brassard  35.0    C  33    6    5   11   -2   20  ...   
9   10.0           Tyler Motte  27.0    C  32    3    6    9    3    4  ...   
10  11.0        Mathieu Joseph  25.0   RW  25    2    7    9    3   12  ...   
11  12.0        Travis Hamonic  32.0

...

New Jersey Devils
      Rk               Player   Age  Pos  GP    G    A  PTS  +/-  PIM  ...  \
0    1.0          Jack Hughes  21.0    C  43   28   26   54   16    2  ...   
1    2.0         Jesper Bratt  24.0    W  43   18   25   43   17    4  ...   
2    3.0        Nico Hischier  24.0    C  42   19   22   41   16    4  ...   
3    4.0      Dougie Hamilton  29.0    D  43   10   27   37   13   30  ...   
4    5.0        Dawson Mercer  21.0    C  43   10   17   27    9   12  ...   
5    6.0          Tomas Tatar  32.0   LW  43   10   17   27   26   18  ...   
6    7.0   Yegor Sharangovich  24.0    C  43   10   10   20    1   10  ...   
7    8.0           Erik Haula  31.0   LW  43    3   17   20    9   33  ...   
8    9.0       Michael McLeod  24.0    C  43    4   14   18    5   30  ...   
9   10.0           Miles Wood  27.0   LW  42    7    9   16   -1   47  ...   
10  11.0          Ryan Graves  27.0    D  40    4   11   15   30   14  ...   
11  12.0    Fabian Zetterlund  23.0    W  34  

Montreal Canadiens
      Rk               Player   Age  Pos  GP    G    A  PTS  +/-  PIM  ...  \
0    1.0          Nick Suzuki  23.0    C  44   16   21   37   -9   16  ...   
1    2.0        Cole Caufield  22.0   RW  44   26   10   36   -9    0  ...   
2    3.0           Kirby Dach  22.0    C  44    8   19   27   -6   32  ...   
3    4.0     Christian Dvorak  26.0    C  44    7   11   18   -9    4  ...   
4    5.0         Sean Monahan  28.0    C  25    6   11   17   -5   16  ...   
5    6.0        Josh Anderson  28.0   RW  42   11    3   14  -13   37  ...   
6    7.0         Kaiden Guhle  21.0    D  36    2   12   14  -14   21  ...   
7    8.0         Arber Xhekaj  22.0    D  42    5    8   13   -4   72  ...   
8    9.0         Mike Hoffman  33.0    C  30    7    5   12   -4   14  ...   
9   10.0      Jonathan Drouin  27.0   LW  28    0   12   12  -14    2  ...   
10  11.0           Jake Evans  26.0    C  43    2    9   11   -8   20  ...   
11  12.0        Jordan Harris  22.0    D  40 

Toronto Maple Leafs
      Rk                   Player   Age  Pos  GP    G    A  PTS  +/-  PIM  \
0    1.0             Mitch Marner  25.0   RW  44   16   36   52    4   16   
1    2.0          Auston Matthews  25.0    C  42   21   27   48   18   14   
2    3.0         William Nylander  26.0    C  44   22   25   47   12   12   
3    4.0             John Tavares  32.0    C  44   20   24   44   -3   24   
4    5.0          Michael Bunting  27.0   LW  44   14   17   31   19   52   
5    6.0           Calle Jarnkrok  31.0    C  37    9   11   20    3   10   
6    7.0        Alexander Kerfoot  28.0    C  44    6   13   19    6   18   
7    8.0            Morgan Rielly  28.0    D  29    0   19   19   -6   11   
8    9.0           Pierre Engvall  26.0   LW  42    9    7   16    6   23   
9   10.0              David Kampf  28.0    C  44    4   11   15    5    4   
10  11.0            Rasmus Sandin  22.0    D  40    3   12   15    7   15   
11  12.0            Mark Giordano  39.0    D  44    1   

Vancouver Canucks
      Rk                Player   Age  Pos  GP    G    A  PTS  +/-  PIM  ...  \
0    1.0      Elias Pettersson  24.0    C  41   18   34   52   11    6  ...   
1    2.0             Bo Horvat  27.0    C  43   30   18   48    5   10  ...   
2    3.0       Andrei Kuzmenko  26.0    F  42   17   21   38    9    2  ...   
3    4.0           J.T. Miller  29.0    C  43   17   21   38  -13   34  ...   
4    5.0          Quinn Hughes  23.0    D  39    3   35   38    6   12  ...   
5    6.0         Ilya Mikheyev  28.0    F  40   12   14   26    3    2  ...   
6    7.0          Brock Boeser  25.0   RW  35    9   17   26  -10   12  ...   
7    8.0         Conor Garland  26.0   RW  42    8   13   21   -8   21  ...   
8    9.0  Oliver Ekman-Larsson  31.0    D  42    1   18   19  -15   16  ...   
9   10.0           Luke Schenn  33.0    D  43    2   12   14    5   58  ...   
10  11.0            Ethan Bear  25.0    D  33    3    7   10    1    8  ...   
11  12.0           Tyler Myers  32

New York Rangers
      Rk             Player   Age  Pos  GP    G    A  PTS  +/-  PIM  ...  \
0    1.0     Artemi Panarin  31.0   LW  44   12   35   47   -4   18  ...   
1    2.0     Mika Zibanejad  29.0    C  44   19   26   45   15    8  ...   
2    3.0           Adam Fox  24.0    D  44    9   35   44   17   16  ...   
3    4.0   Vincent Trocheck  29.0    C  44   13   18   31   -7   26  ...   
4    5.0      Chris Kreider  31.0   LW  42   19   11   30   11   14  ...   
5    6.0     K'Andre Miller  23.0    D  44    5   19   24    2   18  ...   
6    7.0       Filip Chytil  23.0    C  36   12   11   23   10   24  ...   
7    8.0    Barclay Goodrow  29.0   RW  44    8   11   19   -1   34  ...   
8    9.0        Kaapo Kakko  21.0   RW  44    9    9   18    5    8  ...   
9   10.0  Alexis Lafreniere  21.0   LW  43    5   13   18    2   12  ...   
10  11.0        Jimmy Vesey  29.0   LW  43    7    8   15    7    4  ...   
11  12.0       Jacob Trouba  28.0    D  44    3   11   14   -5   37  ..

In [35]:
pg_stats = []
for i in range(len(teams)):
    gpg = teams[i]['G']/teams[i]['GP']
    apg = teams[i]['A']/teams[i]['GP']
    pimpg = teams[i]['PIM']/teams[i]['GP']
    ppppg = teams[i]['PPP']/teams[i]['GP']
    shppg = teams[i]['SHP']/teams[i]['GP']
    spg = teams[i]['S']/teams[i]['GP']
    bpg = teams[i]['BLK']/teams[i]['GP']
    df = pd.DataFrame(list(zip(teams[i]['Player'], teams[i]['Pos'], teams[i]['GP'], gpg, apg, pimpg, ppppg, shppg, spg, bpg)),
                     columns=['Player', 'Pos', 'GP', 'GPG', 'APG', 'PIMPG', 'PPPPG', 'SHPPG', 'SPG', 'BPG'])
    pg_stats.append(df)
    

In [36]:
proj_stats = []

for i in range(len(teams)):
    games = team_count_df.iloc[i]['Games']
    g = pg_stats[i]['GPG']*games
    a = pg_stats[i]['APG']*games
    pim = pg_stats[i]['PIMPG']*games
    ppp = pg_stats[i]['PPPPG']*games
    shp = pg_stats[i]['SHPPG']*games
    s = pg_stats[i]['SPG']*games
    b = pg_stats[i]['BPG']*games
    df = pd.DataFrame(list(zip(teams[i]['Player'], teams[i]['Pos'], teams[i]['GP'], g, a, pim, ppp, shp, s, b)),
                     columns=['Player', 'Pos', 'GP', 'G', 'A', 'PIM', 'PPP', 'SHP', 'S', 'B'])
    proj_stats.append(df)

In [43]:
for i in range(len(teams)):    
    print('\n\n')
    print(team_count_df.iloc[i]['Team'])
    print(proj_stats[i])




Philadelphia Flyers
                 Player  Pos  GP          G          A        PIM       PPP  \
0        Travis Konecny    C  37   3.243243   2.972973   6.891892  1.621622   
1           Kevin Hayes   RW  42   1.190476   3.214286   1.785714  1.428571   
2        Scott Laughton    C  39   1.410256   1.794872   2.051282  1.025641   
3          Joel Farabee   LW  43   1.046512   1.860465   2.674419  0.116279   
4      Anthony DeAngelo    D  38   0.921053   2.368421   5.000000  1.578947   
5          Owen Tippett   RW  38   1.710526   1.447368   1.578947  0.789474   
6          Morgan Frost    C  42   0.952381   1.428571   1.904762  0.357143   
7    James van Riemsdyk   LW  23   1.521739   2.391304   2.608696  0.869565   
8            Noah Cates   LW  43   0.581395   1.395349   0.232558  0.348837   
9        Travis Sanheim    D  43   0.465116   1.279070   3.023256  0.116279   
10        Ivan Provorov    D  43   0.232558   1.511628   1.395349  0.232558   
11         Zack MacEwen    C 

In [76]:
for df in proj_stats:
    df.drop(df[df['Player'] == 'Team Total'].index, inplace = True)
    df.drop(df[df['Pos'] == 'G'].index, inplace = True)
    
print(teams)

[                 Player Pos  GP   G   A  PIM  PPP  SHP    S  BLK
0        Travis Konecny   C  37  24  22   51   12    5  120   26
1           Kevin Hayes  RW  42  10  27   15   12    0  118   14
2        Scott Laughton   C  39  11  14   16    8    6   82   26
3          Joel Farabee  LW  43   9  16   23    1    0   80   26
4      Anthony DeAngelo   D  38   7  18   38   12    0   91   43
5          Owen Tippett  RW  38  13  11   12    6    0   93   27
6          Morgan Frost   C  42   8  12   16    3    0   71   24
7    James van Riemsdyk  LW  23   7  11   12    4    0   50    8
8            Noah Cates  LW  43   5  12    2    3    2   48   35
9        Travis Sanheim   D  43   4  11   26    1    1   68   73
10        Ivan Provorov   D  43   2  13   12    2    1   82  101
11         Zack MacEwen   C  39   4   5   38    0    0   59   22
12         Lukas Sedlak   C  27   3   5   12    0    1   43   27
13          Nick Seeler   D  40   3   5   30    0    1   48   66
14         Cameron York 

In [77]:
players = pd.concat(proj_stats)
players = players.reset_index(drop=True)
players

,Player,Pos,GP,G,A,PIM,PPP,SHP,S,B
0,Travis Konecny,C,37,3.243243,2.972973,6.891892,1.621622,0.675676,16.216216,3.513514
1,Kevin Hayes,RW,42,1.190476,3.214286,1.785714,1.428571,0.000000,14.047619,1.666667
2,Scott Laughton,C,39,1.410256,1.794872,2.051282,1.025641,0.769231,10.512821,3.333333
3,Joel Farabee,LW,43,1.046512,1.860465,2.674419,0.116279,0.000000,9.302326,3.023256
4,Anthony DeAngelo,D,38,0.921053,2.368421,5.000000,1.578947,0.000000,11.973684,5.657895
...,...,...,...,...,...,...,...,...,...,...
853,Max Pacioretty,LW,4,1.500000,0.000000,1.000000,0.500000,0.000000,6.500000,0.500000
854,Dylan Coghlan,D,10,0.000000,0.400000,0.400000,0.000000,0.000000,3.600000,1.000000
855,Jack Drury,C,21,0.000000,0.190476,0.952381,0.000000,0.000000,3.047619,0.857143
856,Ondrej Kase,RW,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [78]:
league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid)

In [79]:
free_agents = [p.name for p in league.free_agents(size=1250)]
# print(free_agents)

In [83]:
fa_projections = players[players['Player'].isin(free_agents)]

In [84]:
fa_projections = fa_projections.sort_values(by=['G'], ascending=False)
fa_projections.head(30)

,Player,Pos,GP,G,A,PIM,PPP,SHP,S,B
281,Eeli Tolvanen,RW,8,2.500000,1.000000,0.000000,1.000000,0.000000,8.500000,4.500000
384,Robby Fabbri,C,5,2.400000,0.000000,1.600000,0.800000,0.000000,8.800000,1.600000
387,Jakub Vrana,LW,2,2.000000,2.000000,0.000000,0.000000,0.000000,10.000000,0.000000
520,Kirill Marchenko,RW,19,1.894737,0.000000,0.842105,0.631579,0.000000,6.105263,2.526316
31,Jake DeBrusk,LW,36,1.777778,1.555556,0.888889,1.222222,0.111111,12.111111,1.555556
5,Owen Tippett,RW,38,1.710526,1.447368,1.578947,0.789474,0.000000,12.236842,3.552632
399,Victor Olofsson,RW,41,1.658537,0.585366,0.390244,0.585366,0.000000,9.170732,0.780488
439,Chris Tierney,C,5,1.600000,0.800000,0.000000,0.000000,0.000000,2.400000,1.600000
269,Daniel Sprong,RW,35,1.600000,1.485714,0.914286,1.028571,0.000000,9.028571,0.685714
137,Lawson Crouse,LW,42,1.523810,0.761905,2.095238,0.285714,0.000000,7.714286,2.952381
